# Web Scraping Genius for Eminem Lyrics

In [2]:
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import ast
import os
import pandas as pd
import time
import requests
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
import matplotlib.pyplot as plt
import seaborn as sns 
import wordcloud
import json

In [ ]:
genius_api_token = 'OOBFnp_z22dvvQbR5A0z6Q3K2kzEjMvnGLwufos0zPw4qC_s-NtSWqVYOVMArDFw'

In [ ]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + genius_api_token}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response
# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print(f'Found {len(songs)} songs by {artist_name}')
    return songs

In [ ]:
# DEMO
request_song_url('Eminem', 300)

In [ ]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    return lyrics

# *WARNING! INAPPROPRAITE CONTENT AHEAD IF YOU RUN CELLS*

In [ ]:
# DEMO
print(scrape_song_lyrics('https://genius.com/Eminem-rap-god-lyrics'))

In [ ]:
def write_lyrics_to_file(artist_name, song_count):
    f = open('lyrics_' + artist_name.lower() + '.txt', 'wb')
    urls = request_song_url(artist_name, song_count)
    for url in urls:
        time.sleep(1)
        lyrics = scrape_song_lyrics(url)
        f.write(lyrics.encode("utf8"))
    f.close()
    num_lines = sum(1 for line in open('lyrics_' + artist_name.lower() + '.txt', 'rb'))
    print(f'Wrote {num_lines} lines to file from {song_count} songs')

In [ ]:
# # ENTIRE SONG DISCOGRAPHY THROUGH GENIUS  
# write_lyrics_to_file('Eminem', 300)

# Lyrics Genius API

## USE API FOR ALL SONGS AND SAVE_LYRICS TO JSON!
## THEN BUILD A FUNCTION TO EXTRACT LYRICS BY SONG FROM JSON

In [243]:
import lyricsgenius
genius_api_token = 'OOBFnp_z22dvvQbR5A0z6Q3K2kzEjMvnGLwufos0zPw4qC_s-NtSWqVYOVMArDFw'
genius = lyricsgenius.Genius(genius_api_token)
genius.verbose = False # Turn off status messages
genius.remove_section_headers = True # Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.skip_non_songs = False # Include hits thought to be non-songs (e.g. track lists)
genius.excluded_terms = ["(Remix)", "(Live)", "(Instrumental)",
                         "Diss-Mas", "'The Equalizer : Soundtrack'",
                        "(A Cappella)", "(Radio Edit)",
                        "(Clean Radio Edit)", "(Skit)",
                        'Freestyle on Beats 1 (Ft. Kurupt)',
                        '(Snippet)'] # Exclude songs with these words in their title
# if lyrics none can we skip or exclude?

In [ ]:
# Maybe use this code to avoid error
# artists = []
# while True:
#     try:
#         artists.append(genius.search_artist(artist, max_songs=10000))
#         break
#     except:
#         pass

In [245]:
# eminem = genius.search_artist("Eminem", max_songs=583, sort="title") # having api search for eminem songs.
# this will automatically have up to 583 songs in eminem object. I should be able to save_lyrics after this

In [246]:
len(eminem.songs)

583

In [90]:
songs = eminem.songs

In [93]:
len(songs)

583

In [94]:
eminem

('Eminem', '583 songs')

In [121]:
eminem.save_lyrics() # saving lyrics of songs to json file

Lyrics_Eminem.json already exists. Overwrite?
(y/n): y
Wrote `Lyrics_Eminem.json`


# Working with json File for Data Framing

In [3]:
with open('./Lyrics_Eminem.json') as json_file:
    data = json.load(json_file)

In [4]:
len(data)

20

In [5]:
len(data['songs'])

583

In [6]:
type(data['songs'][3]['lyrics'])

str

In [7]:
# maybe add album filter to song lyrics function if i only want official album lyrics and songs

# row index will be song names
# 1 column will be lyrics, other columns other info that will be nice for organizing and EDA

In [8]:
data.keys()

dict_keys(['alternate_names', 'api_path', 'description', 'facebook_name', 'followers_count', 'header_image_url', 'id', 'image_url', 'instagram_name', 'is_meme_verified', 'is_verified', 'name', 'translation_artist', 'twitter_name', 'url', 'current_user_metadata', 'iq', 'description_annotation', 'user', 'songs'])

In [9]:
data['songs'][444]['title']

'Seduction'

In [10]:
data['songs'][song_index_integer]['title']

NameError: name 'song_index_integer' is not defined

In [11]:
lyrics_by_song = []
album = []
song_names = [] 
song_index_integer = 2 # skipping the first 3 
for song in data['songs']:
    while song_index_integer != len(data['songs']) - 1:
        song_index_integer = song_index_integer + 1
        song_lyrics = data['songs'][song_index_integer]['lyrics']
        song_title = data['songs'][song_index_integer]['title']
        if song_lyrics is None:
            print(f"Skipped lyrics from song #{song_index_integer}: {data['songs'][song_index_integer]['title']}")
            pass
#         elif data['songs'][song_index_integer]['album']['name'] is None:
#             album.append("Non-Album Release")
#             split_song_lyrics = song_lyrics.split('\n')
#             lyrics_by_song.append(split_song_lyrics)
        else:
            split_song_lyrics = song_lyrics.split('\n')
            #album_name = data['songs'][song_index_integer]['album']['name']
            lyrics_by_song.append(split_song_lyrics)
            song_names.append(song_title)
            print(f"Grabbed lyrics from song #{song_index_integer}: {data['songs'][song_index_integer]['title']}")

len(lyrics_by_song)
len(song_names)
final_dict = dict(zip(song_names, lyrics_by_song))

Grabbed lyrics from song #3: 1996 Underground Freestyle
Grabbed lyrics from song #4: 1997 Freestyle Live at Wetlands, NYC
Grabbed lyrics from song #5: 1997 Rap Olympics
Grabbed lyrics from song #6: 1999
Grabbed lyrics from song #7: 1999 Tim Westwood Freestyle
Grabbed lyrics from song #8: 2004 Tim Westwood Freestyle
Grabbed lyrics from song #9: 2012 Something From Nothing: Art of Rap Freestyle
Grabbed lyrics from song #10: 2.0 Boys
Grabbed lyrics from song #11: 25 To Life
Grabbed lyrics from song #12: 313
Grabbed lyrics from song #13: 3 a.m.
Grabbed lyrics from song #14: 3 Verses
Grabbed lyrics from song #15: 4 Verses
Grabbed lyrics from song #16: 50 Ways
Grabbed lyrics from song #17: 8 Mile
Grabbed lyrics from song #18: 8 Mile: 313 Cypher (Chin Tiki Girls)
Grabbed lyrics from song #19: 8 Mile Background Music
Grabbed lyrics from song #20: 8 Mile: B-Rabbit vs Papa Doc
Grabbed lyrics from song #21: 8 Mile: B-Rabbit vs Supa Emcee
Grabbed lyrics from song #22: 8 Mile: Cheddar Bob Freestyle

In [12]:
len(lyrics_by_song)

565

In [13]:
type(lyrics_by_song)

list

In [14]:
len(song_names)

565

In [15]:
final_dict

{'1996 Underground Freestyle': ['Any rapper saying those kind of rhymes',
  'In this day and age in this period of time',
  'Tryna battle Eminem is worse than',
  'David Starr tryna battle Proof, Eye-Ku, BFlat and Bizarre',
  '(unintelligible)',
  "That's why you beg to differ me, you ain't got no style",
  'Plus you lack delivery, not to brag',
  "I don't need to boast",
  'Look, my face is pale, but you look like you seen a ghost',
  '(unintelligible)',
  "You couldn't slip up in the zone, if I put banana peels",
  'Around this fucking microphone',
  'So get a bulldozer, start from bottom to top',
  'And tear this building down',
  "'cause that's the only way you wrecking shop"],
 '1997 Freestyle Live at Wetlands, NYC': ['Everybody, duck down, all you hear is the sound',
  "Of Slim Shady spittin' rounds at the Lyricist Lounge",
  "What year is this now? I don't know, I forgot",
  "All I know is that I'd die for pot, make the cypher hot",
  'Smoke weed and fuck up my life a lot',
  'G

In [ ]:
# song titles will be columns. lyrics will be contained to 1 row and column intersection
df_without_line_structure.columns = list(df_with_line_structure.index)

In [17]:
df_with_line_structure = pd.DataFrame.from_dict(final_dict, orient='index')

In [24]:
df_with_line_structure

,0,1,2,3,4,5,6,7,8,9,...,808,809,810,811,812,813,814,815,816,817
1996 Underground Freestyle,Any rapper saying those kind of rhymes,In this day and age in this period of time,Tryna battle Eminem is worse than,"David Starr tryna battle Proof, Eye-Ku, BFlat ...",(unintelligible),"That's why you beg to differ me, you ain't got...","Plus you lack delivery, not to brag",I don't need to boast,"Look, my face is pale, but you look like you s...",(unintelligible),...,None,None,None,None,None,None,None,None,None,None
"1997 Freestyle Live at Wetlands, NYC","Everybody, duck down, all you hear is the sound",Of Slim Shady spittin' rounds at the Lyricist ...,"What year is this now? I don't know, I forgot","All I know is that I'd die for pot, make the c...",Smoke weed and fuck up my life a lot,"Give me the money now, bitch",Did you want to see your wife or not?,'Fore I stab her wit' this knife I got,I got a sci-fi plot like a science fiction fli...,I got the Outsidaz up in this bitch,...,None,None,None,None,None,None,None,None,None,None
1997 Rap Olympics,I'ma tell you this for your own benefit,"Your shit was dope as hell, 'specially when yo...",What you need to do is practice on your freest...,"'Fore you come up missin', like Snoop Dogg's p...",This type of literature that I'm spittin' be h...,You gonna swear to God this shit was written,"But it wasn't, I'll kill a dozen rappers when ...","Guzzlin', beat your ass like a jealous husband","Now, you boys are about to suffer and see north",When you go in a coma longer than Vladimir Kon...,...,None,None,None,None,None,None,None,None,None,None
1999,"Hi kids, do you like Kim's bullshit? (No!)",Wanna see me stick her fucking eyeballs right ...,,I got a fucking time traveling machine to go b...,"And see all the piss that she gave me, bitch",She made me fall on my ass,The walls are caving in and I gotta go ham on ...,"Like, woah, so we never fell in love so I didn...",Go through that shit no more,"It's like a nugget, I'm hugging these sluts",...,None,None,None,None,None,None,None,None,None,None
1999 Tim Westwood Freestyle,"Tim Westwood, Marley Marl and Slim Shady and M...","Proof on my side, my sidekick",Y'all can suck my wide -ick,"Said the ""ick"" without the D in front of it",None of you rappers want none of it,I’ll put you on punishment like Big Pun,"Wanna suck my ""ick-ick""? Here's a big one",I'll do it with the fluid that I’m spillin',"I'm illin', plus I'm chillin', I'm not a villain",Forget the controversy surroundin' me,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
You Gon’ Learn,"Pray you see tomorrow little boy, you gon' learn",All them pretty bitches leave yo' ass in the urn,"You ain't sold your last rock, you just better...","But I pray you see tomorrow little boy, you go...","Pray you see tomorrow little boy, you gon' learn",All them pretty bitches leave yo' ass in the urn,"You ain't sold your last rock, you just better...","But I pray you see tomorrow little boy, you go...",,I'm a product of properly hoppin' up out of th...,...,None,None,None,None,None,None,None,None,None,None
You’re Never Over,"The days are cold, livin' without you","The nights are long, I'm growin' older","I miss the days of old, thinkin' about you","You may be gone, but you're never over",,"If Proof could see me now, I know he'd be proud","Somewhere in me deep down, there's something i...","That made him believe in me, now no one can be...","You try, it'll be them doors on Dre's Phantom,...","That means ""suicide,"" homie, you'll never thro...",...,None,None,None,None,None,None,None,None,None,None
Zane Lowe BBC Radio Interview Part 1,"Zane Lowe: Welcome back to Radio One, Maida Va...",Eminem: It's super exciting to be here. I was ...,"ZL: Seeing you with the band, and seeing you p...",E: I think performing has gotten a lot differe...,ZL: Spitting Rap God that just puts everything...,"E: I didn't write that, I freestyled it, 

In [20]:
new_df = df_with_line_structure.agg(lambda x: ' '.join(i for i in x if not pd.isna(i)), axis=1)

In [28]:
new_df = pd.DataFrame(new_df, columns=['lyrics'])

In [29]:
new_df

,lyrics
1996 Underground Freestyle,Any rapper saying those kind of rhymes In this...
"1997 Freestyle Live at Wetlands, NYC","Everybody, duck down, all you hear is the soun..."
1997 Rap Olympics,I'ma tell you this for your own benefit Your s...
1999,"Hi kids, do you like Kim's bullshit? (No!) Wan..."
1999 Tim Westwood Freestyle,"Tim Westwood, Marley Marl and Slim Shady and M..."
...,...
You Gon’ Learn,"Pray you see tomorrow little boy, you gon' lea..."
You’re Never Over,"The days are cold, livin' without you The nigh..."
Zane Lowe BBC Radio Interview Part 1,"Zane Lowe: Welcome back to Radio One, Maida Va..."
Zane Lowe BBC Radio Interview Part 2,"""Who can say for sure? Perhaps a frontal lobot..."


# Importing data, vectorizing and conducting EDA

In [ ]:
cvec = CountVectorizer(stop_words=ENGLISH_STOP_WORDS)
lyrics_cvec = cvec.fit_transform(df['Eminem Lyrics by Line'])

In [ ]:
lyrics_cvec

In [ ]:
lyrics_cvec = pd.DataFrame(lyrics_cvec.todense(), columns = cvec.get_feature_names())

In [ ]:
lyrics_cvec

In [ ]:
lyrics_cvec.sum().sort_values().tail(50).plot.barh(); # checking most popular words across both subreddits
plt.savefig('Most Popular ngram 1,1 Words.png') # not much change in top words. more trump though

In [ ]:
stops = list(ENGLISH_STOP_WORDS) # + ['said', 'say']

wc = wordcloud.WordCloud(width=600, height=400, max_words=50\
                    , stopwords=stops)
text = df['Eminem Lyrics by Line'].str.cat()

wc.generate_from_text(text)
wc.to_image()

In [ ]:
cvec_2_2 = CountVectorizer(ngram_range=(2,2), stop_words=ENGLISH_STOP_WORDS)
lyrics_cvec_2_2 = cvec_2_2.fit_transform(df['Eminem Lyrics by Line'])

In [ ]:
lyrics_cvec_2_2 = pd.DataFrame(lyrics_cvec_2_2.todense(), columns = cvec_2_2.get_feature_names())

In [ ]:
lyrics_cvec_2_2.sum().sort_values().tail(50).plot.barh(); 
plt.savefig('Most Popular ngram 2,2 Words.png')

# BONUS SHPONGLE LYRICS

In [ ]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + genius_api_token}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response
# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print(f'Found {len(songs)} songs by {artist_name}')
    return songs

In [ ]:
# request_song_url('Shpongle', 300)

In [ ]:
def write_lyrics_to_file(artist_name, song_count):
    f = open('lyrics_' + artist_name.lower() + '.txt', 'wb')
    urls = request_song_url(artist_name, song_count)
    for url in urls:
        time.sleep(1)
        lyrics = scrape_song_lyrics(url)
        f.write(lyrics.encode("utf8"))
    f.close()
    num_lines = sum(1 for line in open('lyrics_' + artist_name.lower() + '.txt', 'rb'))
    print(f'Wrote {num_lines} lines to file from {song_count} songs')

In [ ]:
# # ENTIRE SONG DISCOGRAPHY THROUGH GENIUS  
# write_lyrics_to_file('Shpongle', 300)